In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#This script was simply to concatenate data I had downloaded in chunks

In [ ]:
ds1=xr.open_dataset(dir+'era_5_taiwan_xc_5.nc')
u1=ds1.u
ds2=xr.open_dataset(dir+'era_5_taiwan_u_xc_61.nc')
u2=ds2.u
ds3=xr.open_dataset(dir+'era_5_taiwan_u_xc_62.nc')
u3=ds3.u
u_concat=xr.concat([u1,u2,u3],dim='time')
print(u_concat)

In [ ]:
ds1=xr.open_dataset(dir+'era_5_taiwan_v_xc_5.nc')
v1=ds1.v
ds2=xr.open_dataset(dir+'era_5_taiwan_v_xc_61.nc')
v2=ds2.v
ds3=xr.open_dataset(dir+'era_5_taiwan_v_xc_62.nc')
v3=ds3.v
v_concat=xr.concat([v1,v2,v3],dim='time')
print(v_concat)

In [ ]:
ds1=xr.open_dataset(dir+'era_5_taiwan_q_xc_5.nc')
q1=ds1.q
ds2=xr.open_dataset(dir+'era_5_taiwan_q_xc_61.nc')
q2=ds2.q
ds3=xr.open_dataset(dir+'era_5_taiwan_q_xc_62.nc')
q3=ds3.q
q_concat=xr.concat([q1,q2,q3],dim='time')
print(q_concat)

In [ ]:
yrs_neusa=np.arange(1979,2020,1)
for i in range(len(yrs_neusa)):
#for i in range(2):
    year=yrs_neusa[i]
    date_start=dt.datetime(year,5,14,18)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(47*4)]
    if i==0:
        dates_all=date_range
    else:
        dates_all=dates_all+date_range
print(dates_all) 

In [ ]:
u_concat_select=u_concat.sel(time=dates_all)
v_concat_select=v_concat.sel(time=dates_all)
q_concat_select=q_concat.sel(time=dates_all)
print(q_concat_select)

In [ ]:
mflux_x_concat_select=q_concat_select*u_concat_select
mflux_y_concat_select=q_concat_select*v_concat_select

In [ ]:
dk=xr.Dataset()
dk['u']=(('time','level','latitude','longitude'),u_concat_select)
dk['v']=(('time','level','latitude','longitude'),v_concat_select)
dk['q']=(('time','level','latitude','longitude'),q_concat_select)
dk['mflux_x']=(('time','level','latitude','longitude'),mflux_x_concat_select)
dk['mflux_y']=(('time','level','latitude','longitude'),mflux_y_concat_select)
dk.coords['time']=u_concat_select.time
dk.coords['level']=u_concat_select.level
dk.coords['latitude']=u_concat_select.latitude
dk.coords['longitude']=u_concat_select.longitude
dk.to_netcdf(dir+'era_5_taiwan_xc_vars.nc',mode='w',format='NETCDF4')